# 🚀 Despliegue con Streamlit - Factores de anemia

- Miguel Angel Vélez Suarez
- Samuel Pérez Hurtado

In [1]:
#Importamos librerías básicas
import pandas as pd # manipulacion dataframes
import numpy as np  # matrices y vectores
import matplotlib.pyplot as plt #gráfica

In [2]:
#Cargamos el modelo
import pickle
filename = 'data/modelo-hiperparametrizado.pkl'
model,labelencoder,variables,min_max_scaler = pickle.load(open(filename, 'rb'))

In [3]:
# Se instala la librería de streamlit
# !pip install streamlit

In [4]:
#Se crea interfaz gráfica con streamlit para captura de los datos
import streamlit as st

st.title('Predicción de factores de anemia')

age_5yr = st.selectbox('Rango de edad', ['25-29', '30-34', '35-39', '20-24', '15-19', '40-44', '45-49'])
residence = st.selectbox('Tipo de residencia', ['Urban', 'Rural'])
wealth = st.selectbox('Índice de riqueza', ['Richest', 'Richer', 'Middle', 'Poorer', 'Poorest'])
hemoglobin = st.slider('Hemoglobina ajustada a la altura y si es fumador', min_value=18, max_value=160, value=50, step=1)
mosquito_net = st.selectbox('Uso de mosquitero', ['Yes', 'No'])
smokes = st.selectbox('Fumador', ['No', 'Yes'])
breastfeeding = st.slider('Minutos a los que se amamantó el hijo por primera vez', min_value=0, max_value=250, value=10, step=1)
fever = st.selectbox('¿Ha tenido fiebre en las dos últimas semanas?', ['No', 'Yes', "Don't know"])
supplements = st.selectbox('Está tomando suplementos de hierro?', ['No', 'Yes', "Don't know"])

datos = [[age_5yr, residence, wealth, hemoglobin, mosquito_net, smokes, breastfeeding, fever, supplements]]
data = pd.DataFrame(datos, columns=['Age in 5-year groups', 'Type of place of residence',
                                    'Wealth index combined',
                                    'Hemoglobin level adjusted for altitude and smoking (g/dl - 1 decimal)',
                                    'Have mosquito bed net for sleeping (from household questionnaire)',
                                    'Smokes cigarettes', 'When child put to breast',
                                    'Had fever in last two weeks', 'Taking iron pills, sprinkles or syrup']) #Dataframe con los mismos nombres de variables

2023-11-14 00:58:10.929 
  command:

    streamlit run /home/codespace/.local/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [5]:
#Se realiza la preparación
data_preparada=data.copy()
data_preparada = pd.get_dummies(data_preparada, drop_first=False, dtype='int64')
data_preparada[['Hemoglobin level adjusted for altitude and smoking (g/dl - 1 decimal)', 'When child put to breast']]= min_max_scaler.transform(data_preparada[['Hemoglobin level adjusted for altitude and smoking (g/dl - 1 decimal)', 'When child put to breast']])
data_preparada.head()

,Hemoglobin level adjusted for altitude and smoking (g/dl - 1 decimal),When child put to breast,Age in 5-year groups_25-29,Type of place of residence_Urban,Wealth index combined_Richest,Have mosquito bed net for sleeping (from household questionnaire)_Yes,Smokes cigarettes_No,Had fever in last two weeks_No,"Taking iron pills, sprinkles or syrup_No"
0,0.214286,0.044843,1,1,1,1,1,1,1


In [6]:
#Se adicionan las columnas faltantes
data_preparada=data_preparada.reindex(columns=variables,fill_value=0)
data_preparada.head()

,Hemoglobin level adjusted for altitude and smoking (g/dl - 1 decimal),When child put to breast,Age in 5-year groups_20-24,Age in 5-year groups_25-29,Age in 5-year groups_30-34,Age in 5-year groups_35-39,Age in 5-year groups_40-44,Age in 5-year groups_45-49,Type of place of residence_Urban,Wealth index combined_Poorer,Wealth index combined_Poorest,Wealth index combined_Richer,Wealth index combined_Richest,Have mosquito bed net for sleeping (from household questionnaire)_Yes,Smokes cigarettes_Yes,Had fever in last two weeks_No,Had fever in last two weeks_Yes,"Taking iron pills, sprinkles or syrup_No","Taking iron pills, sprinkles or syrup_Yes"
0,0.214286,0.044843,0,1,0,0,0,0,1,0,0,0,1,1,0,1,0,1,0


In [7]:
#Hacemos la predicción
Y_fut = model.predict(data_preparada)
print(labelencoder.inverse_transform(Y_fut))

['Moderate']
